In [43]:
import pandas as pd
import re
import numpy as np

In [2]:
train = pd.read_csv('train.csv','\t')

In [3]:
def parseHTML(part):
    return re.compile(r'<.*?>').sub('', part)

In [4]:
train['description'] = train.apply(lambda row: parseHTML(row['description']), axis=1 )

In [5]:
train['text'] = train['name'] + ' ' + train['description']

In [6]:
all_text = list(train['text'])

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [8]:
cv =CountVectorizer()
m = cv.fit_transform(all_text)

In [9]:
test = pd.read_csv('test.csv', '\t')
test['description'] = test.apply(lambda row: parseHTML(row['description']), axis=1 )

In [10]:
test['text'] = test['name'] + ' ' + test['description']

In [11]:
test.head()

,id,name,description,text
0,200000,Дизайнер-консультант мебели,"Обязанности: Работа с клиентом в салоне,выезд...",Дизайнер-консультант мебели Обязанности: Рабо...
1,200001,Продавец-консультант (ТЦ на Пушкина),Обязанности: ∙ консультирование покупателей по...,Продавец-консультант (ТЦ на Пушкина) Обязаннос...
2,200002,Менеджер по продажам,Торговый Дом «Форт» это ведущая компания Петер...,Менеджер по продажам Торговый Дом «Форт» это в...
3,200003,Продавец-консультант в магазин одежды (ТЦ Волн...,Требуются продавцы консультанты в магазин женс...,Продавец-консультант в магазин одежды (ТЦ Волн...
4,200004,Специалист по охране труда,Обязанности: осуществление контроля по соблю...,Специалист по охране труда Обязанности: осущ...


In [12]:
lr = LogisticRegression()

In [13]:
lr.fit(m, train['target'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
lr

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
def f(x, lr, cv):
    return lr.predict_proba(
        cv.transform([x])
    )[0][1]

In [16]:
test['text'][1]

'Продавец-консультант (ТЦ на Пушкина) Обязанности: ∙ консультирование покупателей по представленным брендам; ∙ поддержание высокого уровня обслуживания покупателей; ∙ поддержание должного уровня презентации брендов в торговом зале.   Требования: ∙ интерес к сфере моды; ∙ хорошие коммуникативные навыки, доброжелательность, активность, позитивность; ∙ опыт работы в продажах приветствуется, но не является обязательным.   Условия: ∙ работа в престижной компании с культовыми брендами одежды; ∙ оформление в соответствии с ТК РФ; ∙ сменный график работы; ∙ заработная плата включает в себя фиксированный оклад и премиальную часть   Дополнительно для наших сотрудников: ∙ корпоративная скидка в магазинах нашей сети; ∙ возможность карьерного и профессионального развития; ∙ обучение внутри компании, участие в корпоративных мероприятиях  '

In [18]:
round(f(test['text'][1], lr, cv))

1.0

In [48]:
res = pd.DataFrame(columns=['id','target'])
for index, row in test.iterrows():
    res = res.append({"id": np.int(row['id']), "target": round(f(row['text'], lr, cv))}, ignore_index=True)

In [54]:
res['id'] = res['id'].astype(int)

In [55]:
res.head()

,id,target
0,200000,1.0
1,200001,1.0
2,200002,1.0
3,200003,1.0
4,200004,0.0


In [44]:
np.int(res['id'][1])

200001

In [63]:
res.to_csv('ready.csv', sep=',', encoding='utf-8',index=False)

In [62]:
predict = lr.predict(train)

ValueError: could not convert string to float: 'Швея Требуется швея на производство мягкой мебели. Желателен опыт работы. на производстве. Знание технологии . Официальное трудоустройства . Полный соц.пакет'

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
fpr, tpr, thresholds = roc_curve(test, predict[:,1])
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1])
plt.ylabel('tpr')
plt.xlabel('fpr')
plt.grid(True)
plt.title('ROC curve')
plt.xlim((-0.01, 1.01))
plt.ylim((-0.01, 1.01))